# Initial EDA on Raw UNGA dataset

Data set is taken from here: https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379

In [71]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import Image
from IPython.core.display import HTML 

from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

from sklearn.preprocessing import scale

%matplotlib inline

In [72]:
df_votes = pd.read_csv('~/Desktop/dataverse_files/UNVotes1-72.tab',sep='\t')

In [73]:
df_votes.member = df_votes.member.fillna(value=0)

In [74]:
df_votes_C = df_votes.drop(['Country', 'Countryname'], axis=1)

# Here goes the important cell:

In [75]:
# creating a df for 1960 alone
df_votes_1960 = df_votes_C[df_votes.year == 1960]

In [76]:
df_votes_1960_members = df_votes_1960[df_votes_1960.member == 1]

In [77]:
df_votes_1960_binary = df_votes_1960[(df_votes_1960.vote <= 1) | (df_votes_1960.vote == 3)]
df_votes_1960_binary.vote.value_counts()
# My data set now shows only yes and no votes in 1960

1    2577
3    1461
Name: vote, dtype: int64

In [78]:
# I want to turn my 1 = yes and 3 = no into 0 = no and 1 = yes
df_votes_1960_binary.vote = df_votes_1960_binary.vote.replace(to_replace=3, value=0, inplace=False, limit=None, regex=False, method='pad', axis=None)
df_votes_1960_binary.head(2)

/Users/jakerattner/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,rcid,ccode,session,member,vote,year
129626,627.0,2,15.0,1.0,1,1960
129627,627.0,20,15.0,1.0,1,1960


In [79]:
features = ['']

In [80]:
df_votes_1960_binary.sort_values('ccode').head()

,rcid,ccode,session,member,vote,year
129626,627.0,2,15.0,1.0,1,1960
134551,652.0,2,15.0,1.0,0,1960
134748,653.0,2,15.0,1.0,1,1960
130217,630.0,2,15.0,1.0,1,1960
134945,654.0,2,15.0,1.0,0,1960


In [81]:
# After doing this model with my current dataframe, I realize this makes no sense.  I have
# a new row for each resolution/country combo.  Instead make the country the record, 
# Then I can plot the country, and the binary on their vote for each resolution is the feature
# I can start by dropping unused columns (keeping ccode)

In [82]:
df_1960_features = df_votes_1960_binary.drop([ 'session', 'year'], axis=1)

In [83]:
df_1960_features.head()

,rcid,ccode,member,vote
129626,627.0,2,1.0,1
129627,627.0,20,1.0,1
129629,627.0,40,1.0,0
129644,627.0,91,1.0,0
129647,627.0,94,1.0,0


In [84]:
df_1960_features['v_res'] = df_1960_features['rcid']*df_1960_features['vote']
df_1960_features.head()

,rcid,ccode,member,vote,v_res
129626,627.0,2,1.0,1,627.0
129627,627.0,20,1.0,1,627.0
129629,627.0,40,1.0,0,0.0
129644,627.0,91,1.0,0,0.0
129647,627.0,94,1.0,0,0.0


In [85]:
df_1960_dummies = pd.concat([df_1960_features,pd.get_dummies(df_1960_features.v_res, prefix='res_')], axis = 1)
df_1960_dummies.head()

,rcid,ccode,member,vote,v_res,res__0.0,res__627.0,res__628.0,res__629.0,res__630.0,...,res__671.0,res__672.0,res__673.0,res__674.0,res__675.0,res__676.0,res__677.0,res__678.0,res__9040.0,res__9041.0
129626,627.0,2,1.0,1,627.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129627,627.0,20,1.0,1,627.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129629,627.0,40,1.0,0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129644,627.0,91,1.0,0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129647,627.0,94,1.0,0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# need to drop res_0.0 column

In [87]:
df_t = df_1960_dummies.groupby('ccode').sum()
df_t.shape
# there were 54 resolutions in 1960
# I'm going to have to do something to make each resultion its own column
# That column will need to look at the vote for the country in the row and assign a binary

(99, 59)

In [88]:
df_t.head()

,rcid,member,vote,v_res,res__0.0,res__627.0,res__628.0,res__629.0,res__630.0,res__631.0,...,res__671.0,res__672.0,res__673.0,res__674.0,res__675.0,res__676.0,res__677.0,res__678.0,res__9040.0,res__9041.0
ccode,,,,,,,,,,,,,,,,,,,,,
2,45332.0,44.0,21,30373.0,23,1,1,0,1,0,...,0,1,0,0,0,0,0,1,1,1
20,45461.0,44.0,27,34397.0,17,1,1,0,1,0,...,0,1,1,1,1,0,0,1,1,1
40,50058.0,51.0,37,41018.0,14,0,0,1,0,1,...,1,0,1,1,1,1,1,0,1,1
41,42150.0,39.0,25,33118.0,14,0,1,0,1,0,...,0,1,0,1,1,1,0,0,1,1
42,31679.0,23.0,16,27172.0,7,0,1,0,1,0,...,0,0,0,1,1,0,0,0,1,1


In [89]:
df_t = df_t.drop(['v_res', 'res__0.0', 'vote','member','rcid'], axis=1)

In [90]:
features = list(df_t)

In [91]:
df_t[features].head()

,res__627.0,res__628.0,res__629.0,res__630.0,res__631.0,res__632.0,res__633.0,res__634.0,res__635.0,res__636.0,...,res__671.0,res__672.0,res__673.0,res__674.0,res__675.0,res__676.0,res__677.0,res__678.0,res__9040.0,res__9041.0
ccode,,,,,,,,,,,,,,,,,,,,,
2,1,1,0,1,0,0,0,1,1,1,...,0,1,0,0,0,0,0,1,1,1
20,1,1,0,1,0,0,0,1,1,1,...,0,1,1,1,1,0,0,1,1,1
40,0,0,1,0,1,1,1,0,0,0,...,1,0,1,1,1,1,1,0,1,1
41,0,1,0,1,0,0,0,1,1,1,...,0,1,0,1,1,1,0,0,1,1
42,0,1,0,1,0,0,0,1,1,1,...,0,0,0,1,1,0,0,0,1,1


In [92]:
df_votes.head(2)

,rcid,ccode,session,member,vote,Country,Countryname,year
0,3.0,2,1.0,1.0,1,USA,United States of America,1946
1,3.0,20,1.0,1.0,3,CAN,Canada,1946


In [93]:
df_t.reset_index(inplace=True)
df_t.head(2)

,ccode,res__627.0,res__628.0,res__629.0,res__630.0,res__631.0,res__632.0,res__633.0,res__634.0,res__635.0,...,res__671.0,res__672.0,res__673.0,res__674.0,res__675.0,res__676.0,res__677.0,res__678.0,res__9040.0,res__9041.0
0,2,1,1,0,1,0,0,0,1,1,...,0,1,0,0,0,0,0,1,1,1
1,20,1,1,0,1,0,0,0,1,1,...,0,1,1,1,1,0,0,1,1,1


In [94]:
df_votes[['ccode','Countryname']].head(2)

,ccode,Countryname
0,2,United States of America
1,20,Canada


In [95]:
df_name_index = pd.DataFrame(df_votes.groupby(['ccode', 'Countryname'])['Countryname'].count())

In [96]:
df_name_index['count'] = df_name_index.Countryname
df_name_index = df_name_index.drop([ 'Countryname'], axis=1)
df_name_index.head(2)

,,count
ccode,Countryname,
2,United States of America,5598
20,Canada,5598


In [97]:
df_name_index.reset_index(inplace=True)
df_name_index.head(2)

,ccode,Countryname,count
0,2,United States of America,5598
1,20,Canada,5598


In [100]:
df_votes_1960 = pd.merge(df_t, df_name_index, on='ccode')
df_votes_1960.head(2)

,ccode,res__627.0,res__628.0,res__629.0,res__630.0,res__631.0,res__632.0,res__633.0,res__634.0,res__635.0,...,res__673.0,res__674.0,res__675.0,res__676.0,res__677.0,res__678.0,res__9040.0,res__9041.0,Countryname,count
0,2,1,1,0,1,0,0,0,1,1,...,0,0,0,0,0,1,1,1,United States of America,5598
1,20,1,1,0,1,0,0,0,1,1,...,1,1,1,0,0,1,1,1,Canada,5598


In [99]:
df_votes_1960[df_votes_1960.cluster == 1]['Countryname'] 

AttributeError: 'DataFrame' object has no attribute 'cluster'

In [ ]:
 df_votes_1960.to_csv('df_votes_1960.csv', index=False)

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
df_PCA_1960 = df_votes_1960.drop(['count', 'Countryname'], axis=1)

In [ ]:
df_PCA_1960.head(2)

In [ ]:
#df_votes_1960.set_index('ccode')
df_PCA_1960.head(2)

In [ ]:
X = df_PCA_1960

pca = PCA()
pca = pca.fit(X)
X

In [ ]:
Z = pca.transform(X)
features_pca = ['PC'+str(i+1) for i in range(pca.n_components_)]
Z = pd.DataFrame(Z, columns=features_pca)
Z.head()

In [ ]:
print(pca.explained_variance_)

In [ ]:
abs((pca.components_))[0]

In [ ]:

print(abs((pca.components_))[0], '\n')
#print('-------------------------------------\n')



In [ ]:
X.reset_index(inplace=True)
X.head()

In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['res__627.0', 'res__628.0', 'res__629.0', 'res__630.0', 'res__631.0', 
                            'res__632.0', 'res__633.0', 'res__634.0', 'res__635.0', 'res__636.0', 
                            'res__637.0', 'res__638.0', 'res__639.0', 'res__640.0', 'res__641.0', 
                            'res__642.0', 'res__643.0', 'res__644.0', 'res__645.0', 'res__646.0', 
                            'res__647.0', 'res__648.0', 'res__649.0', 'res__650.0', 'res__651.0', 
                            'res__652.0', 'res__653.0', 'res__654.0', 'res__655.0', 'res__656.0', 
                            'res__657.0', 'res__658.0', 'res__659.0', 'res__660.0', 'res__661.0', 
                            'res__662.0', 'res__663.0', 'res__664.0', 'res__665.0', 'res__666.0', 
                            'res__667.0', 'res__668.0', 'res__669.0', 'res__670.0', 'res__671.0', 
                            'res__672.0', 'res__673.0', 'res__674.0', 'res__675.0', 'res__676.0', 
                            'res__677.0', 'res__678.0', 'res__9040.0', 'res__9041.0'], axis=1)


In [ ]:
#pd.concat([X,Z])
X = X.join(Z, lsuffix='_caller', rsuffix='_other')
X

In [ ]:
df_PCA_1960 = pd.merge(X, df_PCA_1960, on='ccode')

In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['res__627.0', 'res__628.0', 'res__629.0', 'res__630.0', 'res__631.0', 
                            'res__632.0', 'res__633.0', 'res__634.0', 'res__635.0', 'res__636.0', 
                            'res__637.0', 'res__638.0', 'res__639.0', 'res__640.0', 'res__641.0', 
                            'res__642.0', 'res__643.0', 'res__644.0', 'res__645.0', 'res__646.0', 
                            'res__647.0', 'res__648.0', 'res__649.0', 'res__650.0', 'res__651.0', 
                            'res__652.0', 'res__653.0', 'res__654.0', 'res__655.0', 'res__656.0', 
                            'res__657.0', 'res__658.0', 'res__659.0', 'res__660.0', 'res__661.0', 
                            'res__662.0', 'res__663.0', 'res__664.0', 'res__665.0', 'res__666.0', 
                            'res__667.0', 'res__668.0', 'res__669.0', 'res__670.0', 'res__671.0', 
                            'res__672.0', 'res__673.0', 'res__674.0', 'res__675.0', 'res__676.0', 
                            'res__677.0', 'res__678.0', 'res__9040.0', 'res__9041.0'], axis=1)


In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['index_x','PC3', 'PC4', 'PC5',
       'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14',
       'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23',
       'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32',
       'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41',
       'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50',
       'PC51', 'PC52', 'PC53', 'PC54', 'PC55', 'PC56', 'index_y', 'cluster_y'], axis=1)

In [ ]:
df_PCA_1960.head(2)

In [ ]:
fig = plt.figure(figsize=(9,7))

ax = fig.gca()
ax.scatter(Z.iloc[:,0], Z.iloc[:,1],
                   c=df_PCA_1960.cluster_x)

ax.set_xlabel('principal component 1', fontsize=16)
ax.set_ylabel('principal component 2', fontsize=16)


for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
ax.set_title('PC1 vs PC2\n', fontsize=20)

plt.show()

In [ ]:
df_PCA_1960.head(2)

# Junk and other imports

In [ ]:
# users[users.age == 20]
#
# Select one column from the filtered results.
# users[users.age < 20].occupation
# users[users.age < 20].occupation.value_counts()
# users[(users.age < 20) & (users.gender=='M')]
# users[users.occupation.isin(['doctor', 'lawyer'])]

In [ ]:
df_points = pd.read_csv('~/Desktop/dataverse_files/IdealpointsPublished.tab',sep='\t')
# df points is Ideal point data. Unit of analysis is the country-session 
# (which corresponds closely but not perfectly, to country-year). This 
# contains country-specific ideal point estimates. 

In [ ]:
df_points.columns

In [ ]:
df_dyadic = pd.read_csv('~/Desktop/dataverse_files/Dyadicdata.tab',sep='\t')

In [ ]:
df_dyadic.columns
# Dyadid affinity and ideal point data. Unit of analysis is the dyad-session (year). 
# This file contains distances between country ideal points as well as Affinity scores 
# (although we recommend against using these). 

In [ ]:
df_descriptions = pd.read_csv('~/documents/UN-Capstone/Dscription.csv')

In [ ]:
df_descriptions.tail(30)

In [ ]:
df_descriptions.head()

In [ ]:
df_UNAid = pd.read_csv('~/documents/UN-Capstone/net_aid.csv')

In [ ]:
df_UNAid